TCC Classificação de municípios com base em indicadores do MEI
Importação e Preparação dos dados
28/06/2021

In [2]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
df_mei = pd.read_excel('inadimplencia.xlsx')
df_mei.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5570 entries, 0 to 5569
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   cod_mun        5570 non-null   int64  
 1   nm_municipio   5570 non-null   object 
 2   DAS_pagos      5570 non-null   int64  
 3   qtd_optantes   5570 non-null   int64  
 4   adimplencia    5570 non-null   float64
 5   inadimplencia  5570 non-null   float64
dtypes: float64(2), int64(3), object(1)
memory usage: 261.2+ KB


In [4]:
df_mei.head(20)

,cod_mun,nm_municipio,DAS_pagos,qtd_optantes,adimplencia,inadimplencia
0,1100015,ALTA FLORESTA D'OESTE,612,858,0.7133,0.2867
1,1100023,ARIQUEMES,2420,4512,0.5363,0.4637
2,1100031,CABIXI,87,209,0.4163,0.5837
3,1100049,CACOAL,2412,3997,0.6035,0.3965
4,1100056,CEREJEIRAS,496,757,0.6552,0.3448
5,1100064,COLORADO DO OESTE,611,883,0.6920,0.3080
6,1100072,CORUMBIARA,175,209,0.8373,0.1627
7,1100080,COSTA MARQUES,178,423,0.4208,0.5792
8,1100098,ESPIGAO D'OESTE,880,1073,0.8201,0.1799
9,1100106,GUAJARA-MIRIM,508,1366,0.3719,0.6281


In [9]:
df_mei.drop(columns='nm_municipio', inplace=True)

In [10]:
# IBGE - população
df_populacao = pd.read_excel('populacao.xlsx')
df_populacao.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5570 entries, 0 to 5569
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   uf                      5570 non-null   object
 1   cod_uf                  5570 non-null   int64 
 2   cod_municipio           5570 non-null   int64 
 3   nm_municipio            5570 non-null   object
 4   qtd_populacao_estimada  5570 non-null   int64 
dtypes: int64(3), object(2)
memory usage: 217.7+ KB


In [11]:
df_populacao['cod_mun'] = df_populacao['cod_uf'].apply('{:0>2}'.format) + df_populacao['cod_municipio'].apply('{:0>5}'.format)
df_populacao['cod_mun'] = df_populacao['cod_mun'].astype(int)
df_populacao.head(20)

,uf,cod_uf,cod_municipio,nm_municipio,qtd_populacao_estimada,cod_mun
0,RO,11,15,Alta Floresta D'Oeste,22728,1100015
1,RO,11,23,Ariquemes,109523,1100023
2,RO,11,31,Cabixi,5188,1100031
3,RO,11,49,Cacoal,85893,1100049
4,RO,11,56,Cerejeiras,16204,1100056
5,RO,11,64,Colorado do Oeste,15544,1100064
6,RO,11,72,Corumbiara,7220,1100072
7,RO,11,80,Costa Marques,18798,1100080
8,RO,11,98,Espigão D'Oeste,32695,1100098
9,RO,11,106,Guajará-Mirim,46556,1100106


In [12]:
df_populacao.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5570 entries, 0 to 5569
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   uf                      5570 non-null   object
 1   cod_uf                  5570 non-null   int64 
 2   cod_municipio           5570 non-null   int64 
 3   nm_municipio            5570 non-null   object
 4   qtd_populacao_estimada  5570 non-null   int64 
 5   cod_mun                 5570 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 261.2+ KB


In [13]:
# IBGE - estimativa PIB
df_pib = pd.read_excel('pib.xlsx')
df_pib.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5570 entries, 0 to 5569
Data columns (total 13 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   cod_mun                           5570 non-null   int64  
 1   nm_municipio                      5570 non-null   object 
 2   vlr_adic_bruto_agropecuaria       5570 non-null   int64  
 3   vlr_adic_bruto_industria          5570 non-null   int64  
 4   vlr_adic_bruto_servicos           5570 non-null   int64  
 5   vlr_adic_bruto_administracao      5570 non-null   int64  
 6   vlr_adic_bruto_total              5570 non-null   int64  
 7   impostos_liquidos_sobre_produtos  5570 non-null   int64  
 8   pib                               5570 non-null   int64  
 9   pib_per_capita                    5570 non-null   float64
 10  atividade_maior_vlr_adic_bruto    5570 non-null   object 
 11  atividade_2maior_vlr_adic_bruto   5570 non-null   object 
 12  ativid

In [14]:
# Junção MEI x População
df_completo = pd.merge(df_mei, df_populacao[['cod_mun','uf','qtd_populacao_estimada']], on='cod_mun')
#df_completo[['DAS_pagos','qtd_populacao_estimada']].head()
df_completo.head()

,cod_mun,DAS_pagos,qtd_optantes,adimplencia,inadimplencia,uf,qtd_populacao_estimada
0,1100015,612,858,0.7133,0.2867,RO,22728
1,1100023,2420,4512,0.5363,0.4637,RO,109523
2,1100031,87,209,0.4163,0.5837,RO,5188
3,1100049,2412,3997,0.6035,0.3965,RO,85893
4,1100056,496,757,0.6552,0.3448,RO,16204


In [15]:
# Junção PIB
df_completo = pd.merge(df_completo, df_pib, on='cod_mun')
#df_completo[['DAS_pagos','qtd_populacao_estimada']].head()


In [16]:
df_completo.head(20)

,cod_mun,DAS_pagos,qtd_optantes,adimplencia,inadimplencia,uf,qtd_populacao_estimada,nm_municipio,vlr_adic_bruto_agropecuaria,vlr_adic_bruto_industria,vlr_adic_bruto_servicos,vlr_adic_bruto_administracao,vlr_adic_bruto_total,impostos_liquidos_sobre_produtos,pib,pib_per_capita,atividade_maior_vlr_adic_bruto,atividade_2maior_vlr_adic_bruto,atividade_3maior_vlr_adic_bruto
0,1100015,612,858,0.7133,0.2867,RO,22728,Alta Floresta D'Oeste,165892,26369,123512,155271,471045,28261,499306,21552.47,"Administração, defesa, educação e saúde públic...",Demais serviços,"Pecuária, inclusive apoio à pecuária"
1,1100023,2420,4512,0.5363,0.4637,RO,109523,Ariquemes,163121,339667,1034184,674541,2211513,252260,2463773,23206.36,Demais serviços,"Administração, defesa, educação e saúde públic...",Comércio e reparação de veículos automotores e...
2,1100031,87,209,0.4163,0.5837,RO,5188,Cabixi,62337,5179,25171,42164,134851,5651,140502,25837.12,"Administração, defesa, educação e saúde públic...","Pecuária, inclusive apoio à pecuária","Agricultura, inclusive apoio à agricultura e a..."
3,1100049,2412,3997,0.6035,0.3965,RO,85893,Cacoal,212493,222558,971609,548595,1955256,225198,2180454,25708.96,Demais serviços,"Administração, defesa, educação e saúde públic...",Comércio e reparação de veículos automotores e...
4,1100056,496,757,0.6552,0.3448,RO,16204,Cerejeiras,61973,22818,208728,109796,403315,68863,472179,28714.35,Demais serviços,"Administração, defesa, educação e saúde públic...",Comércio e reparação de veículos automotores e...
5,1100064,611,883,0.6920,0.3080,RO,15544,Colorado do Oeste,73479,26195,103454,105915,309043,21385,330428,20362.88,"Administração, defesa, educação e saúde públic...",Demais serviços,"Pecuária, inclusive apoio à pecuária"
6,1100072,175,209,0.8373,0.1627,RO,7220,Corumbiara,212545,9971,30596,57122,310234,10394,320628,42371.91,"Produção florestal, pesca e aquicultura","Pecuária, inclusive apoio à pecuária","Administração, defesa, educação e saúde públic..."
7,1100080,178,423,0.4208,0.5792,RO,18798,Costa Marques,63525,7218,42875,108861,222479,7684,230164,12890.70,"Administração, defesa, educação e saúde públic...","Pecuária, inclusive apoio à pecuária",Demais serviços
8,1100098,880,1073,0.8201,0.1799,RO,32695,Espigão D'Oeste,120026,61347,183339,194761,559473,48925,608398,18984.55,"Administração, defesa, educação e saúde públic...",Demais serviços,"Pecuária, inclusive apoio à pecuária"
9,1100106,508,1366,0.3719,0.6281,RO,46556,Guajará-Mirim,46931,29036,361929,284832,722728,114154,836882,18279.32,"Administração, defesa, educação e saúde públic...",Demais serviços,Comércio e reparação de veículos automotores e...


In [17]:
df_completo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5570 entries, 0 to 5569
Data columns (total 19 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   cod_mun                           5570 non-null   int64  
 1   DAS_pagos                         5570 non-null   int64  
 2   qtd_optantes                      5570 non-null   int64  
 3   adimplencia                       5570 non-null   float64
 4   inadimplencia                     5570 non-null   float64
 5   uf                                5570 non-null   object 
 6   qtd_populacao_estimada            5570 non-null   int64  
 7   nm_municipio                      5570 non-null   object 
 8   vlr_adic_bruto_agropecuaria       5570 non-null   int64  
 9   vlr_adic_bruto_industria          5570 non-null   int64  
 10  vlr_adic_bruto_servicos           5570 non-null   int64  
 11  vlr_adic_bruto_administracao      5570 non-null   int64  
 12  vlr_ad

In [18]:
# Exportação do dataset completo
df_completo.to_csv('completo.csv', mode='w',          
               index=False , header=True , encoding='utf-8', sep='|',line_terminator='\r\n',float_format='%.2f')

In [19]:
# Verifica valores nulos
#df_completo.isna().all(1)
#for k in range (1,xx)
df_nulos = df_completo[df_completo['vlr_adic_bruto_agropecuaria'].isnull()]
df_nulos

,cod_mun,DAS_pagos,qtd_optantes,adimplencia,inadimplencia,uf,qtd_populacao_estimada,nm_municipio,vlr_adic_bruto_agropecuaria,vlr_adic_bruto_industria,vlr_adic_bruto_servicos,vlr_adic_bruto_administracao,vlr_adic_bruto_total,impostos_liquidos_sobre_produtos,pib,pib_per_capita,atividade_maior_vlr_adic_bruto,atividade_2maior_vlr_adic_bruto,atividade_3maior_vlr_adic_bruto
